# Spike sorting on RTX 5090

10 min recording |

author: laquitainesteeve@gmail.com

Tested on an Ubuntu 24 with a 32GB VRAM Nvidia RTX 5090

Execution time: 16 min

## Setup 

Prerequisites:

1. Activate `demo` environment

2. Enable forward compatibility if your GPU and CUDA libraries are more recent and not supported by editing your matlab `startup.m` file to contain "parallel.gpu.enableCUDAForwardCompatibility(true)" and:

    ```bash
    # manually compile Kilosort3 with CUDA support for forward compatibility
    sudo apt install gcc-11 g++-11 # install gcc 11 compiler
    sudo update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-11 100 # enable temporary
    cd /home/steeve/steeve/epfl/code/spikebias/dataset/01_intermediate/sorters/Kilosort3_buttw_forwcomp/CUDA/
    matlab -batch mexGPUall  # compile matlab mex files
    ```

3. Run notebook ss.run_sorter should work.


You also also run spike sorting manually by replacing SpikeInterface's "run_kilosort.sh" script (created by ss.run_sorter) with and run: 

  ```bash
  #/bin/bash -->
  #manually compile Kilosort3 with CUDA support for forward compatibility
  sudo apt install gcc-11 g++-11 # install gcc 11 compiler
  sudo update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-11 100 # enable temporary

  cd /home/steeve/steeve/epfl/code/spikebias/dataset/01_intermediate/sorters/Kilosort3_buttw_forwcomp/CUDA/
  matlab -batch mexGPUall  # compile matlab mex files

  run Kilosort3 with forward compatibility activated for that session
  cd "/home/steeve/steeve/epfl/code/spikebias/SortingKS3/sorter_output"

  matlab -nosplash -nodisplay -r "parallel.gpu.enableCUDAForwardCompatibility(true); kilosort3_master('/home/steeve/steeve/epfl/code/spikebias/SortingKS3/sorter_output', '/home/steeve/steeve/epfl/code/spikebias/dataset/01_intermediate/sorters/Kilosort3_buttw_forwcomp')"
  ```

In [ ]:
%%time 
%load_ext autoreload
%autoreload 2

# import python packages
import os
from dandi.dandiapi import DandiAPIClient
import spikeinterface.extractors as se
import spikeinterface.sorters as ss
import spikeinterface as si
print("spikeinterface", si.__version__)

# project path
PROJ_PATH = "/home/steeve/steeve/epfl/code/spikebias/"
os.chdir(PROJ_PATH)

# import spikebias package
from src.nodes.sorting import sort_and_postprocess_10m

# recording parameters
REC_SECS = 600 
RECORDING_PATH = "./dataset/00_raw/recording_npx_spont/"

# sorting parameters
SORTER = "kilosort3"
SORTER_PATH = "/home/steeve/steeve/epfl/code/spikebias/dataset/01_intermediate/sorters/Kilosort3_buttw_forwcomp"
SORTING_OUTPUT = "./temp/SortingKS3/" 
SORTER_PARAMS = {
    "detect_threshold": 6,
    "projection_threshold": [9, 9],
    "preclust_threshold": 8,
    "car": True,
    "minFR": 0, # modified
    "minfr_goodchannels": 0, # modified
    "nblocks": 5,
    "sig": 20,
    "freq_min": 300,
    "sigmaMask": 30,
    "lam": 20.0,
    "nPCs": 3,
    "ntbuff": 64,
    "nfilt_factor": 4,
    "do_correction": True,
    "NT": 65792, # modified
    "AUCsplit": 0.8,
    "wave_length": 61,
    "keep_good_only": False,
    "skip_kilosort_preprocessing": False,
    "scaleproc": None,
    "save_rez_to_mat": False,
    "delete_tmp_files": ("matlab_files",),
    "delete_recording_dat": False,
}

# SET KS3 software environment variable
ss.Kilosort3Sorter.set_kilosort3_path(SORTER_PATH)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
spikeinterface 0.101.2
Setting KILOSORT3_PATH environment variable for subprocess calls to: /home/steeve/steeve/epfl/code/spikebias/dataset/01_intermediate/sorters/Kilosort3_buttw_forwcomp
CPU times: user 0 ns, sys: 341 μs, total: 341 μs
Wall time: 300 μs


## Spike sorting

In [ ]:
%%time

# load recording
Recording = si.load_extractor(RECORDING_PATH)
Recording = Recording.frame_slice(start_frame=0, end_frame=Recording.sampling_frequency*REC_SECS)
print('\n', Recording)

# spike sort
Sorting = ss.run_sorter(sorter_name = SORTER,
                        recording = Recording,
                        folder="./temp/SortingKS3",
                        remove_existing_folder = True,
                        verbose = True,
                        **SORTER_PARAMS)

print(Sorting.unit_ids)

In [ ]:
# load old (reference) sorting and compare with sorting on RTX5090
SortingRef = si.load_extractor("/home/steeve/steeve/epfl/code/spikebias/dataset/01_intermediate/sorting/npx_evoked/SortingKS3")
print(len(SortingRef.unit_ids))
print(len(Sorting.unit_ids))

/home/steeve/steeve/epfl/code/spikebias/envs/demo/lib/python3.10/site-packages/spikeinterface/core/base.py:1129: UserWarning: Versions are not the same. This might lead to compatibility errors. Using spikeinterface==0.100.6 is recommended
  warnings.warn(


1660

## Full pipeline

In [3]:
# setup config
CFG = {
    'probe_wiring': {
        'full': {
            'output': 'dataset/00_raw/recording_npx_spont'
        }
    },
    'preprocessing': {
        'full': {
            'output': {
                'trace_file_path': 'dataset/01_intermediate/preprocessing/recording_npx_spont'
            }
        }
    },
    'sorting': {
        'sorters': {
            f"{SORTER}": {
                '10m': {
                    'output': './temp/SortingKS3_10m_RTX5090', #'path/to/sorting/output',
                    'sort_output':'./temp/KS3_output/KS3_output_10m_RTX5090' #'path/to/sorting/sort_output'
                }
            }
        }
    },
    'postprocessing': {
        'waveform': {
            'sorted': {
                'study': {
                    f"{SORTER}": {  # sorter name
                        '10m': './temp/study_ks3_10m_RTX5090' #'path/to/postprocessing/study'
                    }
                }
            }
        }
    }
}

In [ ]:
# sort and postprocess
sort_and_postprocess_10m(CFG,
                         SORTER,
                         SORTER_PARAMS,
                         duration_sec=REC_SECS,
                         is_sort=True,
                         is_postpro=True,
                         copy_binary_recording=True,
                         remove_bad_channels=True)

2025-07-09 18:45:50,705 - root - sorting.py - sort_and_postprocess_10m - INFO - Started sorting 10 minutes recording.
2025-07-09 18:45:50,710 - root - sorting.py - sort - INFO - Removing bad channels...
2025-07-09 18:45:50,714 - root - sorting.py - sort - INFO - Done removing bad channels in: 0.0
2025-07-09 18:45:50,715 - root - sorting.py - sort - INFO - Selected first 10.0 minutes in: 0.0
2025-07-09 18:45:50,716 - root - sorting.py - sort - INFO - Done converting recording as int16 in: 0.0
2025-07-09 18:45:50,716 - root - sorting.py - sort - INFO - Saving int16 binary recording...
write_binary_recording 
n_jobs=32 - samples_per_chunk=400,000 - chunk_memory=154.11 MiB - total_memory=4.82 GiB - chunk_duration=10.00s


write_binary_recording: 100%|██████████| 60/60 [00:09<00:00,  6.13it/s]

2025-07-09 18:46:00,923 - root - sorting.py - sort - INFO - Done copying int16 binary recording in: 10.2
2025-07-09 18:46:00,926 - root - sorting.py - sort - INFO - Start sorting...



/home/steeve/steeve/epfl/code/spikebias/envs/demo/lib/python3.10/site-packages/spikeinterface/core/baserecording.py:815: UserWarning: `file_paths_lenght` is deprecated and will be removed in 0.103.0 please use `file_paths_length`
  warnings.warn(


RUNNING SHELL SCRIPT: /home/steeve/steeve/epfl/code/spikebias/temp/KS3_output/KS3_output_10m_RTX5090/sorter_output/run_kilosort3.sh


                            < M A T L A B (R) >

                  Copyright 1984-2024 The MathWorks, Inc.

                  R2025a (25.1.0.2943329) 64-bit (glnxa64)

                               April 16, 2025



 

To get started, type doc.

For product information, visit www.mathworks.com.

 

Steeve - forward compatibility enabled for more recent GPUs - edit startup.m file to disable

Time   0s. Computing whitening matrix.. 

Getting channel whitening matrix... 


libraries. Compilation can take several minutes. 

> In get_whitening_matrix (line 25)

In preprocessDataSub (line 62)

In kilosort3_master (line 71) 

Channel-whitening matrix computed. 

Time   2s. Loading raw data and applying filters... 

Time  36s. Finished preprocessing 365 batches. 

Drift correction ENABLED

running datashift2...

running extractTemplatesfromSnippets...

running 